In [ ]:
import numpy as np
import importlib, os, datetime
from sus.protocol_designer import System, Protocol, Potential, Compound_Protocol
from sus.protocol_designer.protocol import sequential_protocol
from IPython import display
from IPython.display import HTML, Image
from quick_sim import setup_sim

from edward_tools.coupled_fq_potential import coupled_flux_qubit_non_linear_approx_pot, coupled_flux_qubit_non_linear_approx_force, coupled_flux_qubit_non_linear_approx_pot_break_down

import edward_tools.fq_runner as fq_runner
from edward_tools.visualization import animate_sim_flux_qubit
from edward_tools.initial_state_sampling import extra_constraint_00_and_11_only
from PARAMETER_INPUT import *
from PARAMETER_INPUT import _lambda, _theta, _eta
import importlib

import kyle_tools as kt
import matplotlib.pyplot as plt


from edward_tools import coupled_fq_protocol_library, cfq_runner
from edward_tools import coupled_fq_protocol_library

import edward_tools.cfq_batch_sweep as cfq_batch_sweep
import edward_tools.Analysis_tool.general_analysis_tools as general_analysis_tool
# from edward_tools.Analysis_tool.general_analysis_tools import show_phi_dc_with_time
import edward_tools.Analysis_tool.minimum_value_of_potential as minimum_value_of_potential
from edward_tools.couple_flux_qubit_metrics import fidelityEvaluation


coupled_fq_protocol_library = importlib.reload(coupled_fq_protocol_library)
create_system = coupled_fq_protocol_library.create_system
get_potential_shot_at_different_t = coupled_fq_protocol_library.get_potential_shot_at_different_t
get_potential_shot_at_different_t_1D = coupled_fq_protocol_library.get_potential_shot_at_different_t_1D
create_simple_protocol_parameter_dict = coupled_fq_protocol_library.create_simple_protocol_parameter_dict
coupled_fq_runner = importlib.reload(cfq_runner)
coupled_fq_protocol_library = importlib.reload(coupled_fq_protocol_library)
create_system = coupled_fq_protocol_library.create_system
get_potential_along_a_1D_cutline = coupled_fq_protocol_library.get_potential_along_a_1D_cutline
plotCutlines = coupled_fq_protocol_library.plotCutlines

In [ ]:
# 4. Szilard engine test

# parameters

In [ ]:
has_velocity = True

PHI_0 = 2.067833848 * 1e-15
k_B = 1.38e-23
T = 0.5
k_BT = k_B * T

C_factor = 0.1
L_factor = 0.35
# I_m_factor = 50
# I_m_factor = 15
I_m_factor = 0
time_scale = 1.0


I_p_1, I_p_2 = 2e-6 , 2e-6  # Amp
I_m_1, I_m_2 = 7e-9 * I_m_factor, 7e-9 * I_m_factor                                # Amp
# R_1, R_2 = 1, 1  
R_1, R_2 = 371, 371  
C_1, C_2 = 4e-9 * C_factor, 4e-9 * C_factor                              # F
L_1, L_2 = 1e-9 * L_factor, 1e-9 * L_factor                             # H 

m_c = C_1
m_1 = C_1
m_2 = C_2
x_c = PHI_0 / (2 * np.pi)
time_scale_factor = 1
t_c = np.sqrt(L_1 * C_1)
v_c = x_c / t_c

U0_1 = m_c * x_c**2 / t_c**2 / k_BT
U0_2 = m_2 * x_c**2 / t_c**2 / k_BT
kappa_1, kappa_2, kappa_3, kappa_4 = 1/U0_1, 1/U0_1, 1/U0_1, 1/U0_1

lambda_1 = 2 * np.sqrt(L_1 * C_1) / (C_1 * R_1)
theta_1  = 1
eta_1    = np.sqrt(np.sqrt(L_1 * C_1)/ (R_1 * C_1)) * np.sqrt(2 * kappa_1 / 1**2)

lambda_2 = 2 * np.sqrt(L_1 * C_1) / (C_2 * R_2)
theta_2  = 1 / (C_2/C_1)
eta_2    = np.sqrt(np.sqrt(L_1 * C_1)/ (R_1 * C_1)) * np.sqrt(2 * kappa_2 * (R_1 * C_1**2) / (R_2 * C_2**2))

lambda_3 = 2 * np.sqrt(L_1 * C_1) / (C_1 * R_1)
theta_3  = 4
eta_3    = np.sqrt(np.sqrt(L_1 * C_1)/ (R_1 * C_1)) * np.sqrt(8 * kappa_3)

lambda_4 = 2 * np.sqrt(L_1 * C_1) / (C_2 * R_2)
theta_4  = 4 / (C_2/C_1)
eta_4    = np.sqrt(np.sqrt(L_1 * C_1)/ (R_1 * C_1)) * np.sqrt(8 * kappa_4 * (R_1 * C_1**2) / (R_2 * C_2**2))

gamma = 10


beta_1 = 2 * np.pi * L_1 * I_p_1 / PHI_0; 
beta_2 = 2 * np.pi * L_2 * I_p_2 / PHI_0;

d_beta_1 = 2 * np.pi * L_1 * I_m_1 / PHI_0; 
d_beta_2 = 2 * np.pi * L_2 * I_m_2 / PHI_0;


_lambda = np.array([lambda_1, lambda_2, lambda_3, lambda_4]) 
_theta  = np.array([theta_1, theta_2, theta_3, theta_4])
_eta  =   np.array([eta_1, eta_2, eta_3, eta_4])

In [ ]:
v_1 = np.random.normal(0, np.sqrt(k_BT/m_1)) / v_c
v_2 = np.random.normal(0, np.sqrt(k_BT/m_2)) / v_c
v_3 = np.random.normal(0, np.sqrt(k_BT/(m_1/4))) / v_c
v_4 = np.random.normal(0, np.sqrt(k_BT/(m_2/4))) / v_c

In [ ]:
"""
# step 0: modify parameters
- All the parameters are stored in a separate file PARAMETER_INPUT
- You can override some of the parameters here.
"""
params = {}
params['N'] = 100
params['dt'] = 1/50
params['lambda'] = 1
params['beta'] = 1
params['sim_params'] = [_lambda, _theta, _eta]
params['target_work'] = None
params['monitor_work_dist_in_whole_process'] = True # To monitor the work process
params['applyOffset'] = False
params['comment'] = "Experiment 8 (2024/3/17): 4 well, with no compensation for asym, 1/5000"
params['capacitance'] = [C_1, C_2, C_1/4, C_2/4]
params['mass_special'] = [1, 1, 1/4, 1/4]
params['v_c'] = x_c/t_c
params['k_BT'] = k_BT
params['U0'] = U0_1


In [ ]:
t_c

# define potential

In [ ]:
"""
# step 1: Define potential
"""
coupled_fq_default_param = [1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, x_c]
[phi_1_bound, phi_2_bound, phi_1dc_bound, phi_2dc_bound] = np.array([4, 4, 4, 4])/time_scale_factor

coupled_fq_domain = [[-phi_1_bound, -phi_2_bound, -phi_1dc_bound, -phi_2dc_bound], \
                     [phi_1_bound, phi_2_bound, phi_1dc_bound, phi_2dc_bound]]

# coupled_fq_pot = Potential(coupled_flux_qubit_pot_with_offset_at_00_xy, coupled_flux_qubit_force, 14, 4,\
#                            default_params = coupled_fq_default_param,  relevant_domain = coupled_fq_domain)

coupled_fq_pot = Potential(coupled_flux_qubit_non_linear_approx_pot, coupled_flux_qubit_non_linear_approx_force, 14, 4,\
                           default_params = coupled_fq_default_param,  relevant_domain = coupled_fq_domain)

In [ ]:
"""
# step 2: Define initial condition and protocol
"""
manual_domain=[np.array([-5, -5]), np.array([5, 5])]
phi_1_dcx , phi_2_dcx = 0, 0
phi_1_dc, phi_2_dc = phi_1_dcx, phi_2_dcx

phi_1_x_off, phi_2_x_off = 0, 0
phi_1_dcx_off, phi_2_dcx_off, M_12_off = 0, 0, 0

tilt_factor = 7

if I_m_factor == 0:
    phi_1_x_on = 0
    phi_2_x_on = 0
else:
    phi_1_x_on = -0.046710526315789 * tilt_factor
    phi_2_x_on = -0.046842105263157005 * tilt_factor

phi_1_x_on = -0.1947
phi_2_x_on = -0.1947
phi_2_x_on = -1
# phi_1_x_on = -0
# phi_2_x_on = -0



phi_1_dcx_on = 3.0
phi_2_dcx_on = 3.0
M_12_on = -0.6
    
initial_parameter_dict = {
        "U0_1": U0_1,     "U0_2": U0_2,     "gamma_1": gamma,  "gamma_2": gamma,
        "beta_1": beta_1,   "beta_2": beta_2,   "d_beta_1": d_beta_1 ,   "d_beta_2": d_beta_2,
        "phi_1_x": phi_1_x_off,  "phi_2_x": phi_2_x_off,  "phi_1_dcx": phi_1_dcx_off,  "phi_2_dcx": phi_1_dcx_off,
        "M_12": M_12_off, 'x_c': x_c
}



In [ ]:
zeroDissipation = False
saveAllStates = True
params['sim_params'] = [_lambda, _theta, _eta]
if zeroDissipation:
    params['sim_params'] = [_lambda * 0, _theta, _eta * 0]

params['circuit_parameters'] = {
    "C_factor":C_factor, "L_factor": L_factor, "I_m_factor": I_m_factor, "T": T, 
    "I_p_1": I_p_1, "I_p_2": I_p_2, "I_m_1": I_m_1, "I_m_2": I_m_2,
    "R_1": R_1, "R_2": R_2, "C_1": C_1, "C_2": C_2, "L_1": L_1, "L_2": L_2, 
    "phi_1_x_on": phi_1_x_on, "phi_2_x_on": phi_1_x_on,
    "phi_1_dcx_on": phi_1_dcx_on, "phi_2_dcx_on": phi_2_dcx_on, "M_12_on": M_12_on,
    "gamma": gamma
}
    
TR_initial_condition = [
    (phi_1_dcx_off, phi_2_dcx_off, M_12_off), # t_0
    (phi_1_dcx_off, phi_2_dcx_on,  M_12_off), # t_1
    (phi_1_dcx_off, phi_2_dcx_on,  M_12_on),  # t_2
    (phi_1_dcx_off, phi_2_dcx_off, M_12_on),  # t_3
    (phi_1_dcx_off, phi_2_dcx_off, M_12_off), 
    (phi_1_dcx_on,  phi_2_dcx_off, M_12_off)]

protocol_index = 0

def protocol_maker(time, _p):
    return {
        "duration": time, "name": _p["name"], "phi_1_x": _p["phi_1_x"], "phi_2_x": _p["phi_2_x"],
        "M_12": _p["M_12"], "phi_1_dcx": _p["phi_1_dcx"], "phi_2_dcx": _p["phi_2_dcx"]
    }


_P = [
    {"duration":50 * time_scale, "name":"return to 4 well potential (a)",
        "phi_1_x": phi_1_x_off, "phi_2_x": phi_2_x_off , "M_12": M_12_off,  
        "phi_1_dcx": phi_1_dcx_off, "phi_2_dcx": phi_2_dcx_off,  
    },
    
    # forward
    {"duration": 50 * time_scale, "name":"(t_1) mix in y direction (a)",
        "phi_1_x": phi_1_x_off, "phi_2_x": phi_2_x_on * 0.1, "M_12": M_12_off,  
        "phi_1_dcx": phi_1_dcx_off, "phi_2_dcx": phi_2_dcx_on * 0.5,  
    }, # 1
    {"duration":200 * time_scale, "name":"(t_2) mix in y direction (b)",
        "phi_1_x": phi_1_x_off, "phi_2_x": phi_2_x_on, "M_12": M_12_off,  
        "phi_1_dcx": phi_1_dcx_off, "phi_2_dcx": phi_2_dcx_on,  
    }, # 2
    {"duration":50 * time_scale, "name":"(t_3) conditional tilt",
        "phi_1_x": phi_1_x_off, "phi_2_x": phi_2_x_on, "M_12": M_12_on,  
        "phi_1_dcx": phi_1_dcx_off, "phi_2_dcx": phi_2_dcx_on,  
    }, # 3
    {"duration":50 * time_scale, "name":"(t_4) return to 4 well",
        "phi_1_x": phi_1_x_off, "phi_2_x": phi_2_x_off, "M_12": M_12_off,  
        "phi_1_dcx": phi_1_dcx_off, "phi_2_dcx": phi_2_dcx_off,  
    }, # 4
    
    {"duration":50 * time_scale, "name":"(t_5) conditional tilt",
        "phi_1_x": phi_1_x_on, "phi_2_x": phi_2_x_off, "M_12": M_12_on,  
        "phi_1_dcx": phi_1_dcx_on, "phi_2_dcx": phi_2_dcx_off,  
    }, # 5
    {"duration":50 * time_scale, "name":"(t_6) mix in x direction",
        "phi_1_x": phi_1_x_on, "phi_2_x": phi_2_x_off, "M_12": M_12_off,  
        "phi_1_dcx": phi_1_dcx_on, "phi_2_dcx": phi_2_dcx_off,  
    }, 
    {"duration":200 * time_scale, "name":"return to 4 well potential (a)",
        "phi_1_x": phi_1_x_on * 0.1, "phi_2_x": phi_2_x_off , "M_12": M_12_off,  
        "phi_1_dcx": phi_1_dcx_on * 0.5, "phi_2_dcx": phi_2_dcx_off,  
    },
    {"duration":50 * time_scale, "name":"return to 4 well potential (a)",
        "phi_1_x": phi_1_x_off, "phi_2_x": phi_2_x_off , "M_12": M_12_off,  
        "phi_1_dcx": phi_1_dcx_off, "phi_2_dcx": phi_2_dcx_off,  
    }
]

# bookmark

params['N'] = 50
params['dt'] = 1/500

initial_condition = _P[0]
initial_parameter_dict["phi_1_dcx"], initial_parameter_dict["phi_2_dcx"], initial_parameter_dict["M_12"] = \
initial_condition["phi_1_dcx"], initial_condition["phi_2_dcx"], initial_condition["M_12"]

protocol_list = [
    # protocol_maker(50, _P[1]), protocol_maker(200, _P[2]), 
    # protocol_maker(50, _P[3]),
    protocol_maker(50, _P[4]), 
    # protocol_maker(50, _P[3]), 
    # protocol_maker(50, _P[2]), 
    # protocol_maker(200, _P[1]), protocol_maker(50,  _P[0])
]

# protocol_list = [
    
#     # {"duration":50 * time_scale, "phi_2_x": 0, "phi_2_dcx": 0,  "M_12": -0.6,  "name":"(1) mix in y direction (a)"},    
#     {"duration":50 * time_scale, "phi_2_x": phi_1_x_on * 0.1,  "phi_1_dcx": phi_1_dcx_off, "phi_2_dcx": phi_2_dcx_on * 0.5,  "M_12": M_12_off,  "name":"(1) mix in y direction (a)"},
#     {"duration":200 * time_scale,"phi_2_x": phi_1_x_on, "phi_1_dcx": phi_1_dcx_off, "phi_2_dcx": phi_2_dcx_on,  "M_12": M_12_off,  "name":"(1) mix in y direction (b)"},
#     {"duration":50 * time_scale, "phi_1_dcx": phi_1_dcx_off, "phi_2_dcx": phi_2_dcx_on,  "M_12": M_12_on,   "name":"(2) conditional tilt"},
#     {"duration":50 * time_scale, "phi_2_x": phi_2_x_off, "phi_1_dcx": phi_1_dcx_off, "phi_2_dcx": phi_2_dcx_off, "M_12": M_12_off,  "name":"(3) return to 4 well"},
#     {"duration":50 * time_scale, "phi_1_x": phi_1_x_on, "phi_2_x": phi_1_x_off, "phi_1_dcx": phi_1_dcx_on,  "phi_2_dcx": phi_2_dcx_off, "M_12": M_12_on,  "name": "(4) conditional tilt in x"}, 
#     {"duration":50 * time_scale, "phi_1_dcx": phi_1_dcx_on,  "phi_2_dcx": phi_2_dcx_off, "M_12": M_12_off,  "name":"(5) mix in x direction"}, 
#     {"duration":200 * time_scale, "phi_1_x": phi_1_x_on * 0.1, "phi_1_dcx": phi_1_dcx_on * 0.5, "phi_2_dcx": phi_2_dcx_off, "M_12": M_12_off,  "name":"(6) return to 4 well potential (a)"}, 
#     {"duration":50 * time_scale,  "phi_1_x": phi_1_x_off, "phi_1_dcx": phi_1_dcx_off, "phi_2_dcx": phi_2_dcx_off, "M_12": M_12_off,  "name":"(6) 4 well potential (b)"}, 
# ]



potential_demonstration_list = [
    # protocol_maker(50, _P[1]), protocol_maker(200, _P[2]), 
    # protocol_maker(50, _P[3]),
    protocol_maker(50, {"duration": 50 * time_scale, "name":"(t_1) mix in y direction (a)",
        "phi_1_x": phi_1_x_off, "phi_2_x": phi_2_x_on * 0.1, "M_12": M_12_off,  
        "phi_1_dcx": phi_1_dcx_off, "phi_2_dcx": phi_2_dcx_on * 0.5,  
    }), # 4 well,
    protocol_maker(50, {"duration": 50 * time_scale, "name":"(t_1) mix in y direction (a)",
        "phi_1_x": phi_1_x_off, "phi_2_x": phi_2_x_off, "M_12": M_12_off,  
        "phi_1_dcx": phi_1_dcx_off, "phi_2_dcx": 3.0,  
    }), # lower barrier,
    protocol_maker(50, {"duration": 50 * time_scale, "name":"(t_1) mix in y direction (a)",
        "phi_1_x": phi_1_x_off, "phi_2_x": -1.5, "M_12": M_12_off,  
        "phi_1_dcx": phi_1_dcx_off, "phi_2_dcx": phi_1_dcx_off, 
    }), # lower barrier,
    protocol_maker(50, {"duration": 50 * time_scale, "name":"(t_1) mix in y direction (a)",
        "phi_1_x": phi_1_x_off, "phi_2_x": phi_2_x_off, "M_12": -0.6,  
        "phi_1_dcx": phi_1_dcx_off, "phi_2_dcx": phi_1_dcx_off, 
    }) # lower barrier,
    # protocol_maker(50, _P[3]), 
    # protocol_maker(50, _P[2]), 
    # protocol_maker(200, _P[1]), protocol_maker(50,  _P[0])
]

protocol_list = [
    potential_demonstration_list[3]
]

In [ ]:

"""
# step 3: create the relevant storage protocol and computation protocol
"""
def modifiedFunction(t, result):
    return result


computation_protocol_parameter_dict = coupled_fq_protocol_library.customizedProtocol(initial_parameter_dict, \
                                                                    protocol_list)
storage_protocol, comp_protocol = create_system(computation_protocol_parameter_dict, modifiedFunction = None)

In [ ]:
"""
# step 4: create the coupled_fq_runner
"""

params['capacitance'] = [C_1, C_2, C_1/4, C_2/4]
params['mass_special'] = [1, 1, 1/4, 1/4]
params['v_c'] = x_c/t_c
params['k_BT'] = k_BT

cfqr = cfq_runner.coupledFluxQubitRunner(potential = coupled_fq_pot, params = params, \
                                                storage_protocol= storage_protocol, \
                                                computation_protocol= comp_protocol, \
                                         protocol_list = protocol_list, \
                                        has_velocity=has_velocity)

protocol_time_array, protocol_time_index_array, protocol_all_time_array, protocol_time_all_index_array = cfqr.createProtocolTimeArray(protocol_list, params)
cfqr.initialize_sim()
cfqr.set_sim_attributes()
init_state_saved = cfqr.init_state

In [ ]:
initial_parameter_dict

In [ ]:
cfqr.showPamameterTimeGraph(protocol_time_array, protocol_all_time_array)
# showPamameterTimeGraph(cfqr, protocol_time_array, protocol_all_time_array)

# potential shape check

In [ ]:
import pickle, uuid, glob
# sim_id = uuid.uuid4().hex
# sim_id = "9ad154fe70674d088a00b5b7c1e0a7f5"
comment = "potential_and_parameter_function"
print(sim_id)


In [ ]:
plotLineData = cfqr.plotGraphWithContourGraph(t = 50, cutlineValue=-1.96, cutlineDirection = "v", showGraph=True)

In [ ]:
order = len(glob.glob(f"coupled_flux_qubit_protocol/coupled_flux_qubit_data_gallery/*{comment}*")) + 1
print(order)

In [ ]:
file_name = f"coupled_flux_qubit_protocol/coupled_flux_qubit_data_gallery/{sim_id}_{comment}_order_{order}_plotline_graph.npy"
save = False
save = True
if save:
    np.save(file_name, plotLineData)

### substage

In [ ]:
W_with_offset = [-0.258, -1.880, 2.510, 0.495, -0.386, -2.550, 1.980, 0.230]

In [ ]:
min_potential_at_each_time = np.array([99.89691107926694, 103.91521174619228, 56.54651400870479, 50.856783860072646, 99.89691107926694, 50.856783860072646, 56.5465140089956, 103.91521174619228, 99.89691107926694])
offset_value = min_potential_at_each_time[1:] - min_potential_at_each_time[0:-1]

In [ ]:
W_without_offset = W_with_offset + offset_value

In [ ]:
for x in range(len(W_with_offset)):
    print(f"{x} & {W_without_offset[x]:.3g} & {W_with_offset[x]}\\\\")
    print("\hline")

In [ ]:
for _t in protocol_time_array:
    minimum_data = cfqr.get_minimum_at_for_a_set_of_parameter(_t, initial_parameter_dict, guess = [(-2, -2), (-2, 2), (2, -2), (2, 2)], showPE = True)
    for x in minimum_data:
        print(x)
    print("-"*10)
    # print(minimum_data)

In [ ]:
cutline_direction_array = ["v", "v", "v", "v", "v", "h", "h", "h", "h"]
cutline_value_array = [-1.96439198, -1.96436708, -1.96433431, -2.01268797, -1.96439198,\
                       -2.01268797, -1.96433431, -1.96436708]



In [ ]:
comment = f"2024_7_10_potential_at_each_substage"
sim_id = uuid.uuid4().hex
save = False
# save = True
print(sim_id)
for _i, (_t, _value, _direction) in enumerate(zip(protocol_time_array, cutline_value_array, cutline_direction_array)):
    plotLineData = cfqr.plotGraph(t = _t, cutlineValue=_value, cutlineDirection = _direction, showGraph=True)
    file_name = f"coupled_flux_qubit_protocol/coupled_flux_qubit_data_gallery/{sim_id}_time_{_t}_{comment}_plotline_graph.npy"
    if save:
        np.save(file_name, plotLineData)

In [ ]:
import glob

plotline_graph_array = glob.glob(f"coupled_flux_qubit_protocol/coupled_flux_qubit_data_gallery/{sim_id}_*_plotline_graph.npy")
print(len(plotline_graph_array))

In [ ]:
orderedPlotline_graph = [(np.load(filePath, allow_pickle=True)[()], float(filePath.split("/")[-1].split("_")[2])) for filePath in plotline_graph_array]
orderedPlotline_graph = sorted(orderedPlotline_graph, key = lambda x: x[1])

In [ ]:
for x in plotline_graph_array:
    os.system(f"rm {x}")

In [ ]:
np.load(plotline_graph_array[0], allow_pickle=True)[()]

# actual simulations

In [ ]:
# step 5: perform simulations
manual_domain=[np.array([-5, -5]), np.array([5, 5])]

simResult = cfq_batch_sweep.simulateSingleCoupledFluxQubit(params, initial_parameter_dict, protocol_list, \
                        potential = coupled_fq_pot, potential_default_param = coupled_fq_default_param,            
                        initial_state = init_state_saved, manual_domain = manual_domain, \
                        phi_1_dcx = phi_1_dcx,  phi_2_dcx = phi_2_dcx, \
                        percentage = 1, as_step = np.s_[::100], measure_all_states=saveAllStates, \
                        has_velocity = has_velocity)

In [ ]:
pColor = {"00": "#061DF7", "01": "silver", "10": "#04B3BC", "11": "#FCEF51"}
simResult['cfqr'].showAnimation(protocol_list, computation_protocol_parameter_dict, pColor = pColor, frame_skip = 10)

### Work Analysis

In [ ]:
# %%notify
step_time_array = np.cumsum(np.array([x["duration"] for x in protocol_list]))/params['dt']
name_array = [x["name"] for x in protocol_list]


plt.hist(simResult["work_distribution"], bins = 45)
plt.show()
mean_work = np.mean(simResult["work_distribution"])
jarzyn_term = np.mean(np.exp(-simResult["work_distribution"]))
print("sim_id = ", simResult['simulation_data'])
print(f"N = {params['N']}, dt = {params['dt']}")
print(f"phi_1dcx_on: {phi_1_dcx_on}, phi_2dcx_on: {phi_2_dcx_on}, M_12_on: {M_12_on}")
print(f"L_factor = {L_factor}, C_factor = {C_factor}, I_m_factor = {I_m_factor}, gamma = {gamma}")
print([x["duration"] for x in protocol_list])
print(f'jarzyn = {jarzyn_term}, mean work = {mean_work}')


In [ ]:
import pickle 
save_all_states = True
sim_id = simResult['simulation_data']['simulation_id']

pickle_file_name = f'coupled_flux_qubit_protocol/coupled_flux_qubit_data_gallery/{sim_id}_all_states_step_and_trial_data.pkl'
npy_file_name = f'coupled_flux_qubit_protocol/coupled_flux_qubit_data_gallery/{sim_id}_all_states.npy'
if save_all_states:
    
    all_state_step_and_trial_data = {x: simResult['cfqr'].sim.output.all_state[x] for x in ['step_indices', 'trial_indices']}
    with open(pickle_file_name, 'wb') as f:
        pickle.dump(all_state_step_and_trial_data, f)
    np.save(npy_file_name, simResult['cfqr'].sim.output.all_state['states'])

In [ ]:
with open(f'coupled_flux_qubit_protocol/coupled_flux_qubit_data_gallery/{sim_id}_all_states_step_and_trial_data.pkl', 'rb') as f:
    loaded_all_state_step_and_trial_data = pickle.load(f)


In [ ]:
os.system(f"ls -alh {npy_file_name}")

In [ ]:
os.system(f"rm {npy_file_name}")

In [ ]:
loaded_all_state_step_and_trial_data

In [ ]:
simResult['cfqr'].get_all_state()

In [ ]:
# simResult['cfqr'].getFidelityBarChart()

In [ ]:
# simResult['cfqr'].getFidelityTimeGraph()

In [ ]:
# pColor = {"00": "#061DF7", "01": "#FCEF51", "10": "#3FC7F2", "11": "#F187F4"}
# simResult['cfqr'].showAnimation(protocol_list, computation_protocol_parameter_dict, pColor = pColor, frame_skip= 10)

In [ ]:
# # cutline graph
# # , (1.96, "v","orange")
# pColor = {"00": "#061DF7", "01": "#FCEF51", "10": "#3FC7F2", "11": "#F187F4"}
# importlib.reload(coupled_fq_protocol_library)
# ani,_,_ =  coupled_fq_protocol_library.animate_sim_flux_qubit_with_cutline(simResult['cfqr'], frame_skip = 5, time_array=simResult['cfqr'].sim.target_step_index, 
#                                                                            params = params, cutlineInformation = [(-1.96, "v","red") , (1.96, "v","orange")],
#                                                                           pColor=pColor, opacity = 0, cutlineYLimit = [0, 250])
# video = ani.to_html5_video()
# html = display.HTML(video)
# display.display(html)

In [ ]:
# from edward_tools.visualization import separate_by_state
# # fig, ax = plt.subplots(1, 2, figsize =  [8, 4])
# frame_skip = 10
# plot_distribution_with_cutline(simResult['cfqr'], 0, time_array=protocol_all_time_array, pcolor=pColor, cutlineInformation = [-1.964, "v", "red"])

In [ ]:
# def plot_distribution_with_cutline(self, t, time_array=None, params = None, legend=False, slice_values = None, fig_ax=None, 
#                                    vmin = None, vmax = None, 
#                                    cmap = "afmhot", 
#                                    pcolor = {"00": "blue", "01": "orange", "10": "green", "11": "red"}, 
#                                    cutlineInformation = [-3, "v","red"], **pot_kwargs):
#     names = [r"$\varphi_1$", r"$\varphi_2$"]
#     fig, ax = plt.subplots(1, 2, figsize=[8, 3.5])
#     ax_flatten = ax.flatten()
#     manual_domain=[np.array([-5, -5]), np.array([5, 5])]

#     all_state = self.sim.output.all_state['states']
#     phi_1_dc = all_state[..., 2, 0]
#     phi_2_dc = all_state[..., 3, 0]
#     all_state = all_state[:, :, (0, 1), :]

#     N, nsteps, N_dim = np.shape(all_state)[0], np.shape(all_state)[1], np.shape(all_state)[2]
#     which_axes = [np.s_[..., i, 0] for i in range(N_dim)]
#     x_array = [all_state[item] for item in which_axes]
#     state_lookup = separate_by_state(all_state)
#     opacity = min(1, 300/N)

#     time_array = time_array[::frame_skip]
#     x = x_array[0][:,::frame_skip]
#     y = x_array[1][:,::frame_skip]
#     x_lim = (np.min(x), np.max(x))
#     y_lim = (np.min(y), np.max(y))
#     scat_kwargs = {'alpha':opacity, 'zorder':10}
#     scat = [ax_flatten[0].scatter(x[state_lookup[key], 0], y[state_lookup[key], 0], c = pColor[key], **scat_kwargs) for key in state_lookup]
#     # scat_1 = [ax_flatten[1].scatter(x[state_lookup[key], 0], y[state_lookup[key], 0], **scat_kwargs) for key in state_lookup]

#     modified_manual_domain = [(manual_domain[0][1], manual_domain[0][0]), \
#                               (manual_domain[1][1], manual_domain[1][0])]

# #     resolution = 100
# #     U, X_mesh = self.system.lattice(t, resolution, axes=(0, 1),\
# #                                 manual_domain=modified_manual_domain, slice_values = slice_values)
# #     X, Y = X_mesh[0], X_mesh[1]

# #     # 5. find vmin, vmax, x_min, x_max, y_min and y_max
# #     vmin, vmax = np.min(U), np.max(U)
# #     x_min, x_max = np.min(X), np.max(X)
# #     y_min, y_max = np.min(Y), np.max(Y)
# #     cont = ax_flatten[0].contourf(X, Y, U, 40, vmin = 0, vmax = 400, cmap = "afmhot")


#     X, Y, U, cutlineDirection, _targetAxis, _plotAxis = coupled_fq_protocol_library.get_potential_along_a_1D_cutline(self, t = t, cutlineDirection =cutlineInformation[1], cutlineValue = cutlineInformation[0], resolution = 100, contours = 5, manual_domain = [[-5, -5], [5, 5]])
            
#     coupled_fq_protocol_library.plotCutlines(X, Y, U, cutlineDirection = cutlineDirection, cutlineValue = cutlineInformation[0], cutlineColor = cutlineInformation[2] , contour_plt = ax[0], cutline_plt = ax[1], contours = 5, time = None, graph_title = None, showGraph = True)

    
    

In [ ]:
from edward_tools.visualization import separate_by_state
minimum_point_data = [cfqr.system.get_minimum_point(_t, coupled_flux_qubit_non_linear_approx_pot, guess=[(-2, -2)]) for _t in protocol_time_array]
all_state = simResult['cfqr'].sim.output.all_state['states']
all_state = all_state[:, :, (0, 1), :]
N, nsteps, N_dim = np.shape(all_state)[0], np.shape(all_state)[1], np.shape(all_state)[2]
which_axes = [np.s_[..., i, 0] for i in range(N_dim)]
x_array = [all_state[item] for item in which_axes]
state_lookup = separate_by_state(all_state)
opacity = min(1, 300/N)
x, y = x_array[0], x_array[1]
x_lim = (np.min(x), np.max(x))
y_lim = (np.min(y), np.max(y))
scat_kwargs = {'alpha':opacity, 'zorder':10}

In [ ]:

min_U_along_cutline = []
fig, ax = plt.subplots(5, 4, figsize=[15, 18])
ax_flatten = ax.flatten()
cutlineRange = [-3.5, 3.5]
pColor = {"00": "#061DF7", "01": "#FCEF51", "10": "#3FC7F2", "11": "#F187F4"}
X, Y, U, cutlineDirection, _targetAxis, _plotAxis = get_potential_along_a_1D_cutline(cfqr, t = protocol_time_array[0], cutlineDirection = "v")
plt_line_data_at_t = plotCutlines(X, Y, U, cutlineDirection, cutlineValue = minimum_point_data[0][0][0],\
                        contour_plt=ax_flatten[0], cutline_plt=ax_flatten[1], contours = 20,
                        showGraph=True, time=protocol_time_array[0], cutlineRange = cutlineRange)
min_U = np.min(U)
scat = [ax_flatten[2 * 0].scatter(x[state_lookup[key], protocol_time_index_array[0]], y[state_lookup[key], protocol_time_index_array[0]], c = pColor[key],**scat_kwargs) for key in state_lookup]


for _i in range(1, 5):
    X, Y, U, cutlineDirection, _targetAxis, _plotAxis = get_potential_along_a_1D_cutline(cfqr, t = protocol_time_array[_i], cutlineDirection = "v")
    plt_line_data_at_t = plotCutlines(X, Y, U, cutlineDirection, cutlineValue = minimum_point_data[_i][0][0],\
                            contour_plt=ax_flatten[2*_i], cutline_plt=ax_flatten[2*_i + 1], contours = 20,
                            showGraph=True, time=protocol_time_array[_i], cutlineRange = cutlineRange)
    min_U = np.min(U)
    _index = list(simResult['cfqr'].sim.target_step_index).index(protocol_time_index_array[_i]+1)
    scat = [ax_flatten[2 * _i].scatter(x[state_lookup[key], _index], y[state_lookup[key], _index], c = pColor[key], **scat_kwargs) for key in state_lookup]
    
for _i in range(5, 9):
    X, Y, U, cutlineDirection, _targetAxis, _plotAxis = get_potential_along_a_1D_cutline(cfqr, t = protocol_time_array[_i], cutlineDirection = "h")
    plt_line_data_at_t = plotCutlines(X, Y, U, cutlineDirection, cutlineValue = minimum_point_data[_i][0][1],\
                            contour_plt=ax_flatten[2*_i], cutline_plt=ax_flatten[2*_i + 1], contours = 20,
                            showGraph=True, time=protocol_time_array[_i], cutlineRange = cutlineRange)
    min_U = np.min(U)
    _index = list(simResult['cfqr'].sim.target_step_index).index(protocol_time_index_array[_i]+1)
    scat = [ax_flatten[2 * _i].scatter(x[state_lookup[key], _index], y[state_lookup[key], _index], c = pColor[key], **scat_kwargs) for key in state_lookup]
   

# Fidelity

In [ ]:
simResult["fidelity"]

In [ ]:
def showFidelity(target):

    final_percentage_matrix = np.array([d["final_percentage"] for d in target]).T
    text_box_array = ["\n".join(row) for row in final_percentage_matrix]
    # params_list = list(target["params"])[0]
    fig, ax = plt.subplots(1, 1, figsize = [4, 4])
    # plt.figure(figsize=[4, 4])
    # plt.title(f"N = {params_list['N']}, dt = {params_list['dt']}")
    ax.set_xlim(-5, 5);plt.ylim(-5, 5);
    ax.text(-1.25, -1, "00"); ax.text(-1.25, 4, "01")
    ax.text(4, -1, "10"); ax.text(4, 4, "11")
    ax.text(-3.25, -3.5, text_box_array[0]); ax.text(-3.25, 1.5, text_box_array[1])
    ax.text(1.75, -3.5, text_box_array[2]); ax.text(1.75, 1.5, text_box_array[3])
    ax.vlines(0, ymin = -5, ymax = 5); ax.hlines(0, xmin = -5, xmax = 5)

In [ ]:
# import edward_tools.couple_flux_qubit_metrics as couple_flux_qubit_metrics
# couple_flux_qubit_metrics.fidelityEvaluation
# fidelity_at_t_array = []
# initial_state = simResult['cfqr'].init_state[:, (0, 1), :]
# for t_index in protocol_time_index_array[1:]:
#     state_at_t = simResult['cfqr'].sim.output.all_state["states"][:, t_index, (0, 1), :]
#     fidelity_at_t = couple_flux_qubit_metrics.fidelityEvaluation(initial_state, state_at_t, mapping_state_1_to_state_2_dict)
#     showFidelity(fidelity_at_t)
#     fidelity_at_t_array.append(fidelity_at_t)

# work analysis

In [ ]:
# %%notify
step_time_array = np.cumsum(np.array([x["duration"] for x in protocol_list]))/params['dt']
name_array = [x["name"] for x in protocol_list]


plt.hist(simResult["work_distribution"], bins = 45)
plt.show()
mean_work = np.mean(simResult["work_distribution"])
jarzyn_term = np.mean(np.exp(-simResult["work_distribution"]))
print("sim_id = ", simResult['simulation_data'])
print(f"N = {params['N']}, dt = {params['dt']}")
print(f"phi_1dcx_on: {phi_1_dcx_on}, phi_2dcx_on: {phi_2_dcx_on}, M_12_on: {M_12_on}")
print(f"L_factor = {L_factor}, C_factor = {C_factor}, I_m_factor = {I_m_factor}, gamma = {gamma}")
print([x["duration"] for x in protocol_list])
print(f'jarzyn = {jarzyn_term}, mean work = {mean_work}')


In [ ]:
# step_array = np.arange(simResult["work_statistic"][:,0].shape[0])
step_array = simResult['cfqr'].sim.target_step_index
skip_step = int(len(step_array) * 0.05)
work_mean = simResult["work_statistic"][:,0]
work_std = simResult["work_statistic"][:,1]

plt.errorbar(step_array[::skip_step], work_mean[::skip_step], yerr = work_std[::skip_step])
# plt.yscale('log')

for i, t in enumerate(step_time_array):
    plt.axvline(x = t, color = 'b', linestyle ='--', label = 'axvline - full height')
    # plt.annotate(f"{i+1}", xy=(t, 4.5),  color='red', ha='center', size=16)

print(f'jarzyn = {jarzyn_term}')
print([x["duration"] for x in protocol_list])

In [ ]:

target_time_index = np.array(protocol_time_array)/params['dt']
target_time_index = target_time_index.astype("int") - 1

# work_done_array = [work_mean[i] for i in target_time_index[1:]]
work_done_array = simResult["cfqr"].sim.keyStep_work_statistic[:,0]
work_done_of_each_step = np.array(work_done_array)[1:] - np.array(work_done_array)[:-1]

Measure_Erasue_steps = [0, 1, 2, 3]
Control_Steps = [4, 5, 6, 7]

W_M = work_done_array[4] - work_done_array[0]
W_C = work_done_array[-1] - work_done_array[4]

print("Third way to split the steps: (E, 0-1), (M. 2), (C: 3-6)")
print(f"work_done = {work_done_of_each_step}" )
print(f"measurement + reset = {W_M}, deviation: {W_M - 0.693}")
print(f"control = {W_C}, deviation: {0.693 + W_C}")
print(sum(work_done_of_each_step))

In [ ]:
work_done_array

# animations

In [ ]:
simResult['cfqr'].showAnimation(protocol_list, computation_protocol_parameter_dict)

In [ ]:
from edward_tools.visualization import separate_by_state

In [ ]:
self = simResult['cfqr']
t = 100
resolution = 100

fig, ax = plt.subplots(figsize=(6,6))
protocol_key = ['U0_1', 'U0_2', 'gamma_1', 'gamma_2', 'beta_1', 'beta_2', 'd_beta_1', 'd_beta_2', 'phi_1_x', 'phi_2_x', 'phi_1_dcx', 'phi_2_dcx', 'M_12', 'x_c']
contour_plt = plt


phi_1_dcx_index = protocol_key.index('phi_1_dcx')
phi_2_dcx_index = protocol_key.index('phi_2_dcx')
phi_1_dc_i = self.protocol.get_params(t)[phi_1_dcx_index]
phi_2_dc_i = self.protocol.get_params(t)[phi_2_dcx_index]
slice_values = [0, 0, phi_1_dc_i, phi_2_dc_i]

modified_manual_domain = [(manual_domain[0][1], manual_domain[0][0]), \
                              (manual_domain[1][1], manual_domain[1][0])]
U, X_mesh = self.system.lattice(t, resolution, axes=(0, 1),\
                                    manual_domain=modified_manual_domain, slice_values = slice_values)

# 5. find vmin, vmax, x_min, x_max, y_min and y_max
vmin, vmax = np.min(U), np.max(U)
x_min, x_max = np.min(X), np.max(X)
y_min, y_max = np.min(Y), np.max(Y)

all_state = self.sim.output.all_state['states']
phi_1_dc = all_state[..., 2, 0]
phi_2_dc = all_state[..., 3, 0]
all_state = all_state[:, :, (0, 1), :]

N, nsteps, N_dim = np.shape(all_state)[0], np.shape(all_state)[1], np.shape(all_state)[2]
X, Y = X_mesh[0], X_mesh[1]
which_axes = [np.s_[..., i, 0] for i in range(N_dim)]
x_array = [all_state[item] for item in which_axes]
x = x_array[0]
y = x_array[1]
state_lookup = separate_by_state(all_state, has_velocity = True)
pColor = {"00": "#3FC7F2", "01": "#FCEF51", "10": "#BEDC8C", "11": "#F7B6D2"}
pColor = {"00": "#061DF7", "01": "#FCEF51", "10": "#3FC7F2", "11": "#F7B6D2"}

cont = ax.contourf(X, Y, U, 40, vmin = 0, vmax = 400, cmap = "afmhot")
scat = [ax.scatter(x[state_lookup[key], 120], y[state_lookup[key], 120], c = pColor[key]) for key in state_lookup]

# phi_1dc, phi_2dc and gamma

In [ ]:
general_analysis_tool = importlib.reload(general_analysis_tool)
general_analysis_tool.show_phi_dc_with_time(simResult['cfqr'], params)

# save result

In [ ]:
print(f"current comment: {params['comment']}")
response = input("Are you happy with the current comment? If not, give me a new comment.")
if response == "y":
    pass
else:
    params['comment'] = response

In [ ]:
params['comment'] = "test for save all_states"

In [ ]:
cfq_batch_sweep.saveSimulationResult(simResult, U0_1, timeOrStep = 'step', save = True, save_final_state = False, saveFolderPath = "coupled_flux_qubit_protocol/coupled_flux_qubit_data_gallery", save_all_state = False)

# work_analysis

In [ ]:
selected_work_distribution = simResult["work_distribution"] < -0.2
# selected_work_distribution = simResult["work_distribution"] > 1.5
# selected_work_distribution = np.logical_and(simResult["work_distribution"] > -0.2, simResult["work_distribution"] < 1.5)

In [ ]:
# animations
vmin, vmax = 0, 100
frame_skip = 1000
# phi_1_dc, phi_2_dc = phi_1_dcx, phi_2_dcx
particle_number = 0
selected_particles = simResult["cfqr"].sim.output.all_state["states"][selected_work_distribution, ...]
simResult["cfqr"].system.protocol_list = protocol_list
time_range = (computation_protocol_parameter_dict["t"][0], computation_protocol_parameter_dict["t"][-1])

ani,_,_ = animate_sim_flux_qubit(selected_particles, system = simResult["cfqr"].system ,
                                   times = time_range, frame_skip=frame_skip, color_by_state=True,
                                   vmin = vmin, vmax = vmax,
                                   manual_domain = manual_domain)

In [ ]:
# from IPython.display import HTML
from IPython import display
video = ani.to_html5_video()
html = display.HTML(video)
display.display(html)

In [ ]:
step_time_array = np.array(simResult["cfqr"].sim.output.all_state["step_indices"]) * params['dt']
potential_array = []
for _index, _t in enumerate(step_time_array):
    _params_at_t = simResult['cfqr'].system.protocol.get_params(_t)
    def cfq_potential(state):
        # print(state)
        phi_1, phi_2, phi_1dc, phi_2dc = state
        return coupled_flux_qubit_non_linear_approx_pot(phi_1, phi_2, phi_1dc, phi_2dc, _params_at_t)
    vectorized_cfq_potential = np.vectorize(cfq_potential, signature='(4)->()')
    potential_array.append(vectorized_cfq_potential(selected_particles[:, _index, :, 0]))
    
    

In [ ]:
plt.plot(potential_array)
plt.show()

In [ ]:
protocol_time_array, protocol_time_index_array = cfqr.createProtocolTimeArray(protocol_list, params)
protocol_all_time_index_array = np.array(range(0, int(np.array(protocol_time_array[-1]) / params['dt']) + 1))
protocol_all_time_array  = protocol_all_time_index_array * params['dt']

In [ ]:
minimum_value_data_points = []
importlib.reload(minimum_value_of_potential)
for t in protocol_all_time_array[::1000]:
    minimum_value_data = minimum_value_of_potential.find_minimum_for_all_potential_2(cfqr, t,  beta_1, beta_2, d_beta_1, d_beta_2, verbose = False)
    minimum_value_data_points.append(minimum_value_data[1])

In [ ]:
minimum_value_of_potential = importlib.reload(minimum_value_of_potential)

In [ ]:
minimum_value_data_points = np.array(minimum_value_data_points)

In [ ]:
offset_value = minimum_value_data_points[1:] - minimum_value_data_points[:-1]

In [ ]:
all_minimum_value_at_t = []

for t in protocol_time_array:
    minimum_value_data = minimum_value_of_potential.find_all_minimum_points_for_all_potential(cfqr, t,  beta_1, beta_2, d_beta_1, d_beta_2, verbose = False, guess=[[-2, -2], [-2, 2], [2, -2], [2, 2]])
    all_minimum_value_at_t.append([minimum_value_data, t])

In [ ]:
all_minimum_value_at_t

In [ ]:
plt.plot(minimum_value_data_points)

In [ ]:
plt.plot(protocol_all_time_array[::10], minimum_value_data_points)

In [ ]:
protocol_time_array

# one particle work done

In [ ]:
selected_particles  = simResult["cfqr"].sim.output.all_state["states"][0, :, :, 0]
N_step_number, _ = selected_particles.shape
step_time_array = np.array(simResult["cfqr"].sim.output.all_state["step_indices"]) * params['dt']

In [ ]:
simResult["work_distribution"]

In [ ]:
break_down_potential_categories = ["u1_1", "u2_1",  "u3_1", "u4_1", "u1_2", "u2_2", "u3_2", "u4_2", "u5"]
U_a = np.zeros([N_step_number-1, len(break_down_potential_categories)]) 
U_b = np.zeros([N_step_number-1, len(break_down_potential_categories)] )

In [ ]:
U_a.shape

In [ ]:
def findPotential(cfqr, _t, state):
    _params_at_t = cfqr.system.protocol.get_params(_t)
    def cfq_potential(state):
        phi_1, phi_2, phi_1dc, phi_2dc = state
        return coupled_flux_qubit_non_linear_approx_pot_break_down(phi_1, phi_2, phi_1dc, phi_2dc, _params_at_t)
    return np.array(list(cfq_potential(state).values()))

In [ ]:


for i in range(1, len(step_time_array)):
    # print(step_time_array[i], selected_particles[i, :])
    U_a[i-1] = findPotential(simResult["cfqr"], step_time_array[i], selected_particles[i-1, :])
    # U_a[i-1] = findPotential(simResult["cfqr"], step_time_array[i], selected_particles[i-1, :])
    U_b[i-1] = findPotential(simResult["cfqr"],  step_time_array[i-1], selected_particles[i-1, :])

In [ ]:
u1_1, u2_1, u3_1, u4_1, u1_2, u2_2, u3_2, u4_2, u5 = [U_a[:, i] for i in range(0, 9)]


In [ ]:
# plt.plot(u1_1)
# plt.plot(u2_1)
# plt.plot(u3_1)
# plt.plot(u4_1)


In [ ]:
np.sum(U_a - U_b)

In [ ]:
step_time_array[1:].shape

In [ ]:
selected_particles.shape

In [ ]:
np.zeros((max(100, int(2*NT)), self.potential.N_dim, 2))